# Assignments - module 1

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/vigji/python-cimec/blob/main/assignments/Assignments_1.ipynb)

This notebook contains the assignments to complete for credits for the first module.

**Submission**: Once you're happy with your solutions, send it to me in any form (email the file, share it through Colab/Google Drive, send me a link to your GitHub repo...).

**Deadline**: 15th of July 2023

**Evaluation**: There is no grade, but I will pass assignments that showcase a reasonable degree of understanding og the covered topics. Do your best, and feel free to ask for help if you are struggling!

(Also, try to keep in mind not only the goal of the exercise, but also all the coding best practices we have been considering in the lectures.)

## 0. Spike detection

In this exercise we will be playing around with some (dummy) electrophysiology recordings. Let's start by having a look at the raw data!

In [ ]:
def generate_spike_trace(trace_length=60, firing_rate=1, noise_sigma = 0.05):
    """Function to generate a fake extracellular recording.

    Parameters
    ----------
        trace_length : float
            Duration of the recording in seconds.

        firing_rate : float
            Average firing rate of the neuron in Hz.

        noise_sigma : float
            Noise level.


    Returns:
    --------
        np.array
            Fake extracellular recording.

    """
    np.random.seed(42)
    FS = 10000  # sampling frequency
    n = int(trace_length * FS)  # number of samples

    # Generate spike shape template as a difference of Gaussians.
    # A horrible bunch of magic numbers - do not imitate!
    x = np.arange(30)
    spike_template = np.exp(-(x - 10)**2/6) - np.exp(-(x - 12)**2/16)*0.8

    # Generate spike times from a gaussian distribution:
    spikes_times = np.random.poisson(firing_rate / FS, n)

    # Convolve dirac delta functions of spike times with spike template:
    trace = np.convolve(spikes_times, spike_template)[:n]

    # Add some gaussian noise:
    trace += np.random.normal(0, noise_sigma, n)

    return trace

### Exercise 0.0

Run the function below to generate an synthetic extracellular recording for a neuron. Make a nice plot with the trace; the spikes are the peaks appearing above the noise!

---

(_Optional_) If you want to make a plot with exact x coordinates in seconds, you should know that the trace is sampled at 10000 Hz (10000 points per second).

In [ ]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt

In [ ]:
spike_trace = generate_spike_trace()
sampling_rate = 10000 #Hz

seconds = len(spike_trace)/sampling_rate  #Secondi di recoding

seconds_axisX = np.linspace(0,seconds, num = len(spike_trace))  #x axis
plt.plot(seconds_axisX, spike_trace, 'blue', linewidth = 0.2)

plt.xlabel('Time (s)')
plt.title('synthetic extracellular recording - single neuron')

### Exercise 0.1

Write a function to detect spikes!
The function should take the trace as input, and return the index of each spike as the output (as the index, you should take the position of the spike maximum)

Hint: a good strategy to detect such events is to set a threshold, and look for elements above it. This will not be enough! each spike could have more than 1 point above the threshold, but you want to make sure you take only the spike peak! For this, you will probably need a loop.

Hint: do not start from writing the function. First debug your code running it in a cell, then move it to a function.

Hint: if you want, you can quickly check out the results you are getting by making a scatter plot of the detected spikes overimposed on the electrophysiology trace! (use as x of the dots the indexes of the spikes, and as y the hight of the trace at those indexes)

In [ ]:

def peaks_buster(data):  # LP good name!  :)
  """ Function for finding peaks
      peaks = magnitude of the peak - y value
      idx_peaks = when that peaks happened - x value  """

  THR = 0.25 #threshold
  peaks = []
  idx_peaks = []

  past_point = 0 #

  for idx, value in enumerate(data):

    if idx == 0:
      past_point = 0
    else:
      past_point = data[idx-1]

    actual_point =  value       # datapoint preso in esame ora
    if idx < (len(data)-1):   # controllare che non siamo arrivati alla fine di data
      next_point = data[idx+1]
    else:
      next_point = 0            # ultimo datapoint

    if  (actual_point > THR) & (actual_point > next_point) & (actual_point > past_point) :  #controllare se supera THR ed è effettivamente più alto dei 2 punti vicini
      peaks.append(value)
      idx_peaks.append(idx)

  return peaks, idx_peaks



In [ ]:
peaks, idx_peaks = peaks_buster(spike_trace)

plt.figure(figsize=(10, 5))
plt.plot(spike_trace, 'blue', linewidth = 0.2)
plt.plot(idx_peaks, peaks, 'rD', markersize = 3)

plt.xlabel('Time (s)')
plt.legend(['spike', 'peaks'],bbox_to_anchor=(1.05, 1), loc='upper left')

### Exercise 0.2

We now want to have a look at the shape of those spikes. For this, we will create a function that crops small chunks of the trace around each spike peak.

Write a `crop_event()` function that takes as inputs:
   - the recording array
   - the spike indexes
   - a `n_points_pad` variable specifying the number of points to include before and after the spike

And returns a matrix of shape `(n_spikes, n_points*2)` containing the trace chunks cropped around spike events!

Hint: A good strategy coult be initialize an empty matrix and then fill it in a loop with the trace around the spikes.

**This function can be very useful in many contexts!** You can use it every time you want to crop a timeseries around events (e.g., EEG data or video kinematics data around some stimuli). So keep it at hand in the future!

---

(_Optional_) Pro challenge: Try to do it without for loops! if you construct a matrix with the indexes of the points you want to exctract from the trace, you can use it directly to index the trace!
For indexing in this way, you want to build a matrix that looks like this:
```
array([[...t0-2, t0-1, t0, t0+1, t0+2...],
       [...t1-2, t1-1, t1, t1+1, t1+2...],
       [...t2-2, t2-1, t2, t2+1, t2+2...],])
```
Where `t0`, `t1`, `t2`... are the indexes of each spike, and you take as many points before and after as specified by the `n_points_pad` paramenter.

Building this matrix without loops is not trivial but it can be done nicely with numpy broadcasting!

In [ ]:
n_points_pad = 3

def crop_event(spike_trace, idx_peaks, n_points_pad):
  """ spike_trace = data to crop
      idx_peaks = when the peaks happened
      n_points_pad = how many data points after and before each peak you want to crop
  """

  real_n_points_pad = (n_points_pad*2)+1    #se n_points_pad = 3, vuol dire da -3 a +3, a step di 1, quindi 7 step
  chunks_peaks = np.zeros((len(idx_peaks), (n_points_pad*2))) #matrice vuota

  points_close_peaks_float = np.linspace(n_points_pad , (n_points_pad*-1), num = real_n_points_pad)
  points_close_peaks = points_close_peaks_float.astype(int)  #conversione da float ad integer

  #Se non voglio far comparire nel crop anche il valore originale, devo rimuovere lo zero da "points_close_peaks"
  mask_zero = (points_close_peaks!=0)  #mask per selezionare tutto ciò che non è zero
  points_close_peaks = points_close_peaks[mask_zero]  #rimuovere zero
  #print(points_close_peaks)

  for idx , value_spike in enumerate(idx_peaks):
    for idx_point , value_point in enumerate(points_close_peaks):
      spike_to_grab = (value_spike - value_point)
      chunks_peaks[idx, idx_point] = spike_trace[spike_to_grab]

      #print(spike_to_grab) #i 3 spike precedenti e poi i 3 che seguono

  return(chunks_peaks)

In [ ]:
cropped_data = crop_event(spike_trace,idx_peaks,3)
cropped_data

### Exercise 0.3

Finally, make two subplots one close to the other. On the left, use `plt.matshow` to show the spike matrix. On the right,
plot each individual spike (rows of the matrix) using `plt.plot` with gray lines, and the average spike shape in red on top.

---

(Optional) If you want you can try to normalize the matrix before plotting by subtracting the average of each row (as we were doing for the daily temperatures)!

In [ ]:
# LP: Something's funny - Idk why but you seem to not always find maxima, otherwise peaks should always be at the same
# position in the plot!

fig, axes = plt.subplots(1, 2, figsize = (10, 8)) # create two subplots

# primo subplot
im = axes[0].matshow(cropped_data)  #spike matrix
cbar = fig.colorbar(im, ax=axes[0])
axes[0].set(title = 'spike matrix')

im.set_cmap('coolwarm')
max_value_spike= np.max(np.abs(cropped_data))
im.set_clim(vmin=-max_value_spike, vmax=max_value_spike)  #mettiamo lo stesso range per la colormap

# second subplot
for row in cropped_data:
  axes[1].plot(row, color = 'grey')   # individual spike

column_means = np.mean(cropped_data, axis = 0)
axes[1].plot(column_means, color = 'r')   # mean spike
axes[1].set(title = 'individual and mean spike')


## 1. Real books data

After having appreciated how many books the universe of all possible books contains, let's now focus just on the reachable ones - and how much people like them!

Here, we will download the information about about thousands volumns available on Amazon. Just a tiny fraction of Babel's books, but way more organized!

We will also get a dataset of users writing reviews, and a dataset of reviews.

### Exercise 1.0

Using, `pandas`, read the `.csv` files containing the books, the ratings, and the user data that you can find at the  urls defined below.

Then, plot an histogram of all the ratings from all users, and another histogram with the age of the users:

In [ ]:
users_df_url = "https://github.com/vigji/python-cimec/raw/main/assignments/files/users.csv"
ratings_df_url = "https://github.com/vigji/python-cimec/raw/main/assignments/files/ratings.csv"
books_df_url = "https://github.com/vigji/python-cimec/raw/main/assignments/files/books.csv"


In [ ]:
users = pd.read_csv(users_df_url)
ratings = pd.read_csv(ratings_df_url)
books = pd.read_csv(books_df_url)

fig, ax = plt.subplots(1, 2, figsize = (10, 6))

#Ratings
ax[0].hist(ratings['rating'], color = 'seagreen')
ax[0].set(xlabel = "score", title = "Ratings")

#Age
ax[1].hist(users['age'] , color = 'royalblue')
ax[1].set(xlabel = "year", title = "Age")

### Exercise 1.1

Using the ratings dataframe, compute the average rating for each book, and count how many reviews each book had. Then:
- find out which book had the highest number of reviews.
- find out which book had the highest average rating - but include only books that have at least 100 reviews!


Finally, look for the titles that correspond to those book codes (ISBNs are unique book codes).

In [ ]:
book_counts = ratings['ISBN'].value_counts()

#highest number of review
N_review = book_counts.max()
most_reviewed_ISBN = book_counts.idxmax()

#Given ISBN code, find the title in books dataframe
mask = books['ISBN']==most_reviewed_ISBN
# LP: most times (always?) values returns arrays, we're to make peace with that...
title_most_reviewed_book = books.loc[mask , 'title'].values[0]  #Non ho ben capito perchè debba usare anche .values[0] in modo da selezionare solo il titolo. Se non lo faccio title_most_reviewed_book mi riporta anche la riga del dataframe in cui c'è quel codice ISBN

print(f"The most reviewed book is '{title_most_reviewed_book}', with {N_review} reviews. Its ISBN code is {most_reviewed_ISBN} ")

In [ ]:
avg_rating = ratings.groupby('ISBN').mean()

# LP: kind of what I'm reading now maybe
#Top book with highest avg rating - we don't care about "mattone polacco minimalista di scrittore morto suicida giovanissimo" with less then 100 reviews
mask_book_100 = book_counts.loc[book_counts > 100]  # Good but define this magic number!

# LP: 
# LP: Super-duper important!! You gave me the example for next iteration of the course of why NEVER 
# using iloc unless you are ABSOLUTELY sure of what you are doing (aka, don't use it):
# It assumes that your dataframes are sorted in exactly the same way, but they not always are!
# In this case, you have found a mask over one set, but to apply it you should mask semantical indexes (ISBN codes)
# NOT absolute indexes!
# Compare your result with what you get with 

# book_100 = avg_rating.loc[mask_book_100.index]

# And see that the result will look more plausible :)

book_100 = avg_rating.iloc[mask_book_100]

highest_avg_rating = book_100['rating'].max()
highest_avg_rating_ISBN = book_100['rating'].idxmax()

#Given ISBN code, find the title in books dataframe - same code as above
mask2 = books['ISBN']==highest_avg_rating_ISBN
title_top_book = books.loc[mask2 , 'title'].values[0]

print(f"The book with the highest average is '{title_top_book}'. Its ISBN code is {highest_avg_rating_ISBN} ")

In [ ]:
avg_rating

### Exercise 1.2

Let's get even more specific! Let's find the preferences of users in specific countries and with different ages.

Use the users DataFrame to select only italian users under 40 years old. Then, go back to the reviews dataframe and filter only reviews from those users. Compute the average ratings (include only books that have at least 3 reviews) and sort the ISBNs by average rating. Finally, find the books corresponding to each ISBN code to get which books got the best ratings in this coort of people!

(_Optional_): from the users DataFrame generate a list of all the countries present in the dataset. Then, find the highest rated book in each one of those countries.

In [ ]:
#Italians under 40
mask_young_italians = (users['country'] == "italy") & (users['age'] < 40)
young_italians = users.loc[mask_young_italians ]

#Italians' ratings under 40
mask_italians_ID = ratings['user_id'].isin(young_italians['user_id'])
ratings_italians_all = ratings[mask_italians_ID]

N_reviews_ita_all = ratings_italians_all['ISBN'].value_counts()   #libri con codice ISBN e corrispondenti review da italians
N_reviews_ita = N_reviews_ita_all.loc[N_reviews_ita_all > 2]   #teniamo solo i libri con almeno 3 review

mask = ratings_italians_all['ISBN'].isin(N_reviews_ita.index)
ratings_italians = ratings_italians_all.loc[mask]   #singole review, tenendo però solo le review per libri con almeno 3 review. Ho già detto review?


In [ ]:
avg_ratings_italians = ratings_italians.groupby('ISBN').mean()  #media recensioni per ogni libro
ranking_books = avg_ratings_italians.sort_values(by= 'rating' , ascending=False) #Dal più alto al più basso in termini di media

#Dal dataframe books prendere i libri che corrispondono al codice ISBN
# LP: here you filter differently, and you actually don't encounter the issue above
mask_books = books['ISBN'].isin(ranking_books.index)
top_book_italy = books.loc[mask_books, :]

top_book_italy

In [ ]:
# LP: No foos!!!
foo_df = pd.merge(top_book_italy, ranking_books , on = 'ISBN') #Uniamo il dataframe dei top book italy, con i rispettivi rating

#Voglio vedere anche quante recensioni ha ogni singolo libro
# LP: good check!
df_N_review = pd.DataFrame({'N_review':N_reviews_ita})  #prendi "N_review_ita" e convertilo in un dataframe
df_N_review['ISBN'] = df_N_review.index   #prendi i suoi index (che sono i codici ISBN) e aggiungilo in una nuova colonna di value

#Findal df con libri, relativa media recensioni, e numero di review
# LP: In my pandas version I have to drop an ISBN column here.
final_df = pd.merge(foo_df, df_N_review , on = 'ISBN')
final_df = final_df.drop (columns = ['ISBN' , 'user_id'])  #Facciamo un pò di pulizia
final_df = final_df.sort_values(by= 'rating' , ascending=False)
final_df